<center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma">Vision</div></center>
<img src="./output.png" alt="Study.vision" style="width: 200px;"/>
<h1><center><div style="direction:rtl;font-family:B Lotus, B Nazanin, </div></center></h1>

## <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma"></div>


<div style="direction:rtl;text-align:right;font-family:Tahoma">

<br/>
 a
</div>

http://dataset.class.vision/rnn/RNN-Video-6action.zip

<br/>
<div style="direction:rtl;text-align:right;font-family:Tahoma">
    
</div>

<strong>UCF-101</strong>
[https://www.crcv.ucf.edu/data/UCF101.php](https://www.crcv.ucf.edu/data/UCF101.php)






In [1]:
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model
from keras.layers import Input
import numpy as np
import os.path
from tqdm import tqdm
import csv
import random
import glob
import os.path
import sys
import operator
import threading
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array, load_img

C:\Users\User\AppData\Roaming\Python\Python38\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\numpy\.libs\libopenblas.jpijnswnnan3ce6lli5fwsphut2vxmth.gfortran-win_amd64.dll
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
seq_length= 40
max_frames = 300
image_shape=(224, 224, 3)
base_path = "D:/dataset/RNN-Video"


In [3]:
with open(os.path.join('D:/dataset/RNN-Video/data_file_5class.csv'), 'r') as fin:
    reader = csv.reader(fin)
    data = list(reader)

PermissionError: [Errno 13] Permission denied: 'D:/dataset/RNN-Video/data_file_5class.csv'

In [ ]:
train_path = os.path.join(base_path, 'train')
classes =os.listdir(train_path)
classes = sorted(classes)
classes

<div style="direction:rtl;text-align:right;font-family:Tahoma">
  a.
</div>

In [4]:
data_clean = []
for item in data:
    if int(item[3]) >= seq_length and int(item[3]) <= max_frames:
        data_clean.append(item)

NameError: name 'data' is not defined

In [5]:
len(data_clean)

0

In [6]:
def get_n_sample_from_video(sample, seq_length):
    path = os.path.join(base_path, sample[0], sample[1])
    filename = sample[2]
    images = sorted(glob.glob(os.path.join(path, filename + '*jpg')))

    #Given a list and a size, return a rescaled/samples list. For example,
    #if we want a list of size 5 and we have a list of size 25, return a new
    #list of size five which is every 5th element of the origina list.
    # Get the number to skip between iterations.
    skip = len(images) // seq_length

    # Build our new output.
    output = [images[i] for i in range(0, len(images), skip)]

    # Cut off the last one if needed.
    return output[:seq_length]

In [7]:
data_clean[3]

IndexError: list index out of range

In [8]:
len(get_n_sample_from_video(data_clean[3], 40))

IndexError: list index out of range

In [9]:
# Get model with pretrained weights.
base_model = InceptionV3(weights='imagenet', include_top=True)

# We'll extract features at the final pool layer.
model = Model(inputs=base_model.input,
        outputs=base_model.get_layer('avg_pool').output)

def model_predict(image_path):
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Get the prediction.
    features = model.predict(x)
    return features[0]

96116736/96112376 [==============================] - 10s 0us/step


In [12]:
os.makedirs('sequences', exist_ok=True)
for video in tqdm(data_clean):

    # Get the path to the sequence for this video.
    path = os.path.join('sequences', video[2] + '-' + str(seq_length) + \
        '-features')  # numpy will auto-append .npy

    # Check if we already have it.
    if os.path.isfile(path + '.npy'):
        continue

    # Get the frames for this video.
    frames = get_n_sample_from_video(video, seq_length)

    # Now loop through and extract features to build the sequence.
    sequence = []
    for frame in frames:
        features = model_predict(frame)
        sequence.append(features)

    # Save the sequence.
    np.save(path, sequence)

100%|████████████████████████████████████████████████████████████████████████████████| 439/439 [18:31<00:00,  2.85s/it]
